In [274]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import IndexSlice as idx

In [275]:
techlog_raw = pd.read_csv("csv_data_files\TECHLOG.csv")
techlog_raw['DATE'] = pd.to_datetime(techlog_raw['DATE'], format="%Y-%m-%d")
techlog_raw['ATA'] = techlog_raw['ATA'].astype("category")

util_raw = pd.read_csv("./csv_data_files/UTIL.csv")
util_raw['TO_DATETIME'] = pd.to_datetime(util_raw['TO_DATE'] + ' ' + util_raw['TO_TIME (UTC)'])
util_raw['LAND_DATETIME'] = pd.to_datetime(util_raw['LAND_DATE'] + ' ' + util_raw['LAND_TIME (UTC)'])
util_raw['FL_DURR'] = util_raw['LAND_DATETIME'] - util_raw['TO_DATETIME']
util_raw.drop(columns=['TO_DATE', 'TO_TIME (UTC)', 'LAND_DATE', 'LAND_TIME (UTC)'], inplace=True)

In [276]:
util_raw['CYC'].sum()

177

In [277]:
fluids = techlog_raw[['AC_REG', 'DATE', 'ENG1_OIL', 'ENG2_OIL', 'APU_OIL', 'IDG1_OIL','G_HYD','B_HYD','Y_HYD']].dropna()
fluids_grouped = fluids.groupby(['AC_REG',pd.Grouper(key='DATE',freq='3D')]).sum()

In [278]:
hrs = util_raw.groupby(['AC_REG', pd.Grouper(key='TO_DATETIME', freq='3D',)])[['FL_DURR']].sum()
hrs['FL_DURR'] = hrs['FL_DURR']/np.timedelta64(1,'s')/3600

In [279]:
eng1_consum= fluids_grouped[fluids_grouped['ENG1_OIL']>0][['ENG1_OIL']]

In [280]:
list = []
for reg in eng1_consum.index.get_level_values(0).unique():
    hrs_per_reg = hrs.loc[reg].asfreq('3D', method='ffill')
    for date in eng1_consum.loc[reg].index.get_level_values(0).unique():
        list.append(eng1_consum.loc[idx[reg,date]]['ENG1_OIL'] / hrs_per_reg.loc[date]['FL_DURR'])

eng1_consum['ENG1_OIL_QTZ/FH'] = list
eng1_consum

ENG1_OIL  ENG1_OIL_QTZ/FH
AC_REG DATE                                 
PK-PWA 2022-04-21       2.0         0.394737
       2022-04-24       1.0         0.165746
PK-PWC 2022-04-09       1.0         0.062305
       2022-04-21       1.0         0.062305
       2022-04-24       2.0         0.109991
       2022-05-06       1.0         0.126050
       2022-05-15       2.0         0.285036
       2022-05-30       1.0         0.166667
       2022-06-05       1.0         0.166205
       2022-06-08       2.0         0.321716
       2022-06-11       2.0         0.303797
       2022-06-17       1.0         0.136674
       2022-06-23       4.0         0.178571
       2022-06-26       2.0         0.093023